In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
import warnings
warnings.filterwarnings("ignore")
import statsmodels.formula.api as smf

In [2]:
# colnames = ['day','order_id','item_det_info','pay_timestamp','buyer_id','promise_speed','if_cainiao','merchant_id','Logistics_review_score']
data = pd.read_csv("../replication/mic_bin.csv").drop(columns = ['Unnamed: 0','level_0']).rename(columns = {'fraction':'action_time','Days':'day_count'})
data.head()

,order_id,order_date,logistic_order_id,action,facility_id,facility_type,city_id,logistic_company_id,timestamp,day,...,facility_count,week,day_values,day_of_week,recieve_count,arrive_count,depart_count,scan_count,consign_count,post_action_time
0,100534527,20170115,53756568.0,CONSIGN,NaN,NaN,NaN,532.0,2017-01-16 08:49:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
1,100534527,20170115,53756568.0,SIGNED,112694.0,NaN,NaN,532.0,2017-01-19 16:51:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
2,100534527,20170115,53756568.0,GOT,NaN,NaN,NaN,532.0,2017-01-16 18:00:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
3,100534527,20170115,53756568.0,DEPARTURE,NaN,NaN,NaN,532.0,2017-01-17 06:54:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
4,100534527,20170115,53756568.0,ARRIVAL,NaN,NaN,NaN,532.0,2017-01-17 06:25:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264


In [3]:
data.columns

Index(['order_id', 'order_date', 'logistic_order_id', 'action', 'facility_id',
       'facility_type', 'city_id', 'logistic_company_id', 'timestamp', 'day',
       'item_det_info', 'pay_timestamp', 'buyer_id', 'promise_speed',
       'if_cainiao', 'merchant_id', 'Logistics_review_score', 'min', 'max',
       'action_time', 'day_count', 'avg_action_time', 'action_count',
       'facility_count', 'week', 'day_values', 'day_of_week', 'recieve_count',
       'arrive_count', 'depart_count', 'scan_count', 'consign_count',
       'post_action_time'],
      dtype='object')

In [4]:
df = data[['order_id','logistic_company_id','week','merchant_id','avg_action_time','Logistics_review_score']].drop_duplicates()
df.head(10)

,order_id,logistic_company_id,week,merchant_id,avg_action_time,Logistics_review_score
0,100534527,532.0,3.0,513,0.307430,5.0
6,100534527,NaN,3.0,513,0.307430,5.0
7,60175737,247.0,2.0,419,0.633054,5.0
16,60175737,NaN,2.0,419,0.633054,5.0
17,34518067,273.0,4.0,236,0.499853,5.0
23,34518067,NaN,4.0,236,0.499853,5.0
24,111264400,247.0,1.0,286,0.428563,5.0
34,111264400,NaN,1.0,286,0.428563,5.0
35,9815812,273.0,1.0,286,0.293557,5.0
41,9815812,NaN,1.0,286,0.293557,5.0


In [5]:
df = df.dropna(axis=0,subset = ["logistic_company_id"])

In [6]:
df

,order_id,logistic_company_id,week,merchant_id,avg_action_time,Logistics_review_score
0,100534527,532.0,3.0,513,0.307430,5.0
7,60175737,247.0,2.0,419,0.633054,5.0
17,34518067,273.0,4.0,236,0.499853,5.0
24,111264400,247.0,1.0,286,0.428563,5.0
35,9815812,273.0,1.0,286,0.293557,5.0
...,...,...,...,...,...,...
212303,77554025,532.0,27.0,298,0.574117,5.0
212312,127327610,532.0,27.0,323,0.532375,5.0
212322,117994338,132.0,30.0,238,0.445255,5.0
212330,1587221,741.0,29.0,328,0.506725,5.0


# Regression on avg_action_time(x) Logistics_review_score(y)
## entire data

In [7]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['avg_action_time']
X_train = df[cols]
y_train = df['Logistics_review_score']

# We must add an intercept as the standard model doesn't automatically fit one 1s
X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Logistics_review_score   R-squared:                       0.001
Model:                                OLS   Adj. R-squared:                  0.001
Method:                     Least Squares   F-statistic:                     29.33
Date:                    Sat, 20 Nov 2021   Prob (F-statistic):           6.16e-08
Time:                            09:57:52   Log-Likelihood:                -23848.
No. Observations:                   25676   AIC:                         4.770e+04
Df Residuals:                       25674   BIC:                         4.772e+04
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
co

# top shipper (logistic_company_id)

In [8]:
shipper = df['logistic_company_id'].value_counts()
top_shippers = list(shipper.index)[:5]
top_shippers

[247.0, 149.0, 565.0, 532.0, 674.0]

In [9]:
df_top_shipper = df.query("logistic_company_id in [247.0, 149.0, 565.0, 532.0, 674.0]")
df_top_shipper

,order_id,logistic_company_id,week,merchant_id,avg_action_time,Logistics_review_score
0,100534527,532.0,3.0,513,0.307430,5.0
7,60175737,247.0,2.0,419,0.633054,5.0
24,111264400,247.0,1.0,286,0.428563,5.0
42,106714580,247.0,5.0,286,0.640701,5.0
49,37183351,247.0,3.0,25,0.322391,5.0
...,...,...,...,...,...,...
212272,107562263,674.0,27.0,236,0.388734,5.0
212278,92155832,247.0,29.0,139,0.740741,5.0
212284,51233420,247.0,30.0,213,0.485593,5.0
212303,77554025,532.0,27.0,298,0.574117,5.0


# top weeks (week)

In [10]:
weeks = df['week'].value_counts()
top_weeks = list(weeks.index)[:5]
top_weeks

[25.0, 2.0, 22.0, 6.0, 7.0]

In [11]:
df_top_weeks = df.query("week in [25.0, 2.0, 22.0, 6.0, 7.0]")
df_top_weeks

,order_id,logistic_company_id,week,merchant_id,avg_action_time,Logistics_review_score
7,60175737,247.0,2.0,419,0.633054,5.0
68,110202042,247.0,2.0,387,0.535392,5.0
83,86449234,247.0,2.0,425,0.397953,5.0
144,117265081,674.0,2.0,367,0.325510,5.0
152,118966572,143.0,2.0,194,0.793424,5.0
...,...,...,...,...,...,...
191491,97925386,143.0,25.0,140,0.634068,5.0
191509,79755612,431.0,25.0,65,0.575756,5.0
191519,129914465,431.0,25.0,360,0.525886,5.0
191528,121413955,149.0,22.0,286,0.422643,5.0


# top Merchant (merchant_id)

In [12]:
merchants = df['merchant_id'].value_counts()
top_merchants = list(merchants.index)[:5]
top_merchants

[286, 225, 65, 107, 139]

In [13]:
df_top_merchants = df.query("merchant_id in [286, 225, 65, 107, 139]")
df_top_merchants

,order_id,logistic_company_id,week,merchant_id,avg_action_time,Logistics_review_score
24,111264400,247.0,1.0,286,0.428563,5.0
35,9815812,273.0,1.0,286,0.293557,5.0
42,106714580,247.0,5.0,286,0.640701,5.0
57,80040990,532.0,1.0,139,0.546699,5.0
125,135550561,565.0,1.0,225,0.518187,5.0
...,...,...,...,...,...,...
212134,130158208,532.0,29.0,225,0.513321,1.0
212159,17623350,532.0,30.0,65,0.477497,5.0
212175,53429871,132.0,26.0,65,0.488383,5.0
212247,2387159,674.0,29.0,65,0.444385,5.0


# Regression on avg_action_time(x) Logistics_review_score(y)
## top shippers, weeks, merchants 

In [14]:
def reg(df, col = 'avg_action_time'):
    col = col
    X_train = df[col]
    y_train = df['Logistics_review_score']
    

    # We must add an intercept as the standard model doesn't automatically fit one 1s
    X_train = sm.add_constant(X_train)

    # fit the data to the model
    model = sm.OLS(y_train, X_train).fit()
    return [round(model.params[col],3), round(model.pvalues[1],3)]
#     return model.summary()

In [15]:
reg(df_top_shipper)

[0.158, 0.0]

In [16]:
# shippers
param = []

for i in top_shippers:
    df = df_top_shipper[df_top_shipper['logistic_company_id'] == i]
    param.append(reg(df))

df1 = pd.DataFrame(param, index = ['n=1','n=2','n=3','n=4','n=5'], columns=['shipper_coefficient ', 'shipper_p_value'])

In [17]:
df1

,shipper_coefficient,shipper_p_value
n=1,0.159,0.006
n=2,0.237,0.000
n=3,0.075,0.407
n=4,0.156,0.166
n=5,-0.029,0.812


In [18]:
# weeks
param = []

for i in top_weeks:
    df = df_top_weeks[df_top_weeks['week'] == i]
    param.append(reg(df))

df2 = pd.DataFrame(param, index = ['n=1','n=2','n=3','n=4','n=5'], columns=['week_coefficient ', 'week_p_value'])

In [19]:
# merchants
param = []

for i in top_merchants:
    df = df_top_merchants[df_top_merchants['merchant_id'] == i]
    param.append(reg(df))

df3 = pd.DataFrame(param, index = ['n=1','n=2','n=3','n=4','n=5'], columns=['merchant_coefficient ', 'merchant_p_value'])

In [20]:
df_top = pd.concat([df1,df2,df3], axis = 1)
df_top

,shipper_coefficient,shipper_p_value,week_coefficient,week_p_value,merchant_coefficient,merchant_p_value
n=1,0.159,0.006,0.263,0.035,0.102,0.351
n=2,0.237,0.000,0.286,0.068,0.124,0.310
n=3,0.075,0.407,0.109,0.386,0.216,0.094
n=4,0.156,0.166,0.501,0.002,0.263,0.127
n=5,-0.029,0.812,0.315,0.067,-0.038,0.766


Most coefficients are positive, higher avg_action_time -> higher delivery score  
p of top merchants' coefficients are not very significant